# Numerische Überlegungen zur Summenbildung

Wir verwenden 32-Bit Floats (Single Precision Floating Point) siehe [hier](https://en.wikipedia.org/wiki/Single-precision_floating-point_format).

Der folgende Code erstellt ein Array mit 1 Million und 1 Eintrag:
- 1 Million mal die 1.0
- 1 mal die 1,000,000,000,000.0 (1e12)

Das Array `values_asc` ist aufsteigend sortiert. Es wird ebenso eine absteigend sortierte Version `values_desc` und eine zufällig angeordnete Version `values_rand` erstellt. 

Von allen Arrays wird mit der `.sum()` -Methode die Summe gebildet. Welche werten kommen raus? Warum unterscheiden sich diese? Danach kommt eine manuelle For-Schleife - wie vergleicht sich das Ergebnis. Können Sie in NumPy-Dokumentation rausfinden welches Verfahren hier genutzt wird?

Implementieren Sie zwei geeignetere Verfahren:
- Verwendung einer 64-Bit Float Variable für die Summe
- [Paarweise Addition](https://en.wikipedia.org/wiki/Pairwise_summation)

Welche Ergebnisse erhalten Sie? Warum sind diese genauer?

In [1]:
import numpy as np

In [2]:
# Erstelle ein Array mit 1 Million mal 1.0 und 1 Eintrag 1e12
values_asc = np.array(int(1e6) * [1.0] + [1e12],dtype=np.float32)
values_asc

array([1.e+00, 1.e+00, 1.e+00, ..., 1.e+00, 1.e+00, 1.e+12], dtype=float32)

In [3]:
# Setze den Eintrag für 1e12 an die erste Stelle
values_desc = np.flip(values_asc)
values_desc

array([1.e+12, 1.e+00, 1.e+00, ..., 1.e+00, 1.e+00, 1.e+00], dtype=float32)

In [4]:
# Zufällige Reihenfolge
np.random.seed(2021)
values_rand = values_asc.copy()
np.random.shuffle(values_rand)
values_rand

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [5]:
# Summen:
print(f'Ascending order:  {values_asc.sum():18,.0f}')
print(f'Random order:     {values_rand.sum():18,.0f}')
print(f'Descending order: {values_desc.sum():18,.0f}')

Ascending order:   1,000,000,978,944
Random order:      1,000,000,389,120
Descending order:    999,999,995,904


In [8]:
values_desc[0]  + values_desc[1]

1000000000000.0

In [9]:
total = np.float32(0)
for v in values_desc:
    total += v
print(f'For-loop desc:    {total:18,.0f}')

For-loop desc:       999,999,995,904


In [10]:
total = np.float64(0)
for v in values_desc:
    total += v
print(f'For-loop desc:    {total:18,.0f}')

For-loop desc:     1,000,000,995,904


In [98]:
a = 1e20
a

1e+20

In [100]:
b = 1
b

1

In [101]:
a + b

1e+20

In [13]:
values_asc[:-1].sum()

1000000.0

In [31]:
values_f16 = np.array(int(1e6) * [0.001],dtype=np.float16)
values_f16

array([0.001, 0.001, 0.001, ..., 0.001, 0.001, 0.001], dtype=float16)

In [32]:
values_f16.sum()

991.0

In [34]:
total = np.float16(0)
for v in values_f16:
    total += v
print(f'For-loop desc:    {total:18,.5f}')

For-loop desc:               4.00000


In [35]:
total

4.0

In [36]:
total + values_f16[0]

4.0

In [26]:
def pairwise(arr):
    if len(arr) == 0:
        return 0
    elif len(arr) == 1:
        return arr[0]
    else:
        mid = int(len(arr) / 2)
        return pairwise(arr[:mid]) + pairwise(arr[mid:])

In [39]:
%%timeit
pairwise(values_rand)

898 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
pairwise(values_f16)

1001.0

In [43]:
def pairwise_partial(arr):
    if len(arr) <= 64:
        return arr.sum()
    else:
        mid = int(len(arr) / 2)
        return pairwise(arr[:mid]) + pairwise(arr[mid:])

In [44]:
%%timeit
pairwise_partial(values_rand)

926 ms ± 76.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
tmp = np.array([1,2,3,4,5])
mid = len(tmp) / 2

In [23]:
mid = int(mid)
mid

2

In [24]:
tmp[:mid]

array([1, 2])

In [25]:
tmp[mid:]

array([3, 4, 5])